In [ ]:
##### Assignmnet - 3 
##### Name - ROHAN RAJKUMAR KOLEWAD
##### PRN no - 230350125066

# Importing Important libraries 

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd
import missingno as msno
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
water =  pd.read_csv('water_potability.csv')

## Performing EDA on Selected Dataset

In [ ]:
water.shape

In [ ]:
water.head()

In [ ]:
water.describe()

In [ ]:
water.dtypes

In [ ]:
water.info()

### Dependent Variable Analysis

In [ ]:
water_potability = water["Potability"].map({0: "Not Potable", 1: "Potable"})
dependent_V = pd.DataFrame(water_potability.value_counts()).reset_index()
dependent_V.columns = ["Potability", "Count"]

fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(dependent_V["Count"], labels=dependent_V["Potability"], autopct="%.1f%%", startangle=90)
ax.set_title("Pie Chart of Potability Feature")
ax.axis("equal")
ax.legend(loc="best", bbox_to_anchor=(1, 0.5))
plt.show()

### Correlation Between Features

In [ ]:
water_corr= water.select_dtypes(include='number').columns
corr= water[water_corr].corr()
corr
plt.subplots(figsize=(12, 8))
sns.heatmap(corr, annot=True)

### Plotting Boxplots for all numerical attributes.

In [ ]:
columns = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate',
           'Conductivity', 'Organic_carbon', 'Trihalomethanes',
           'Turbidity',]

# Create a figure with subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

# Plot each boxplot on a separate subplot
for ax, column in zip(axes.flatten(), columns):
    sns.boxplot(data=water, x=column, ax=ax)
    ax.set_title(column)
    
    
plt.tight_layout()
plt.show()

### Distribution plots and Dispersion patterns

In [ ]:
print(water.columns)

In [ ]:
non_potable = water.query("Potability == 0")
potable = water.query("Potability == 1")

plt.figure(figsize = (15,15))
for ax, col in enumerate(water.columns[:9]):
    plt.subplot(3,3, ax + 1)
    plt.title(col)
    sns.kdeplot(x = non_potable[col], label = "Non Potable")
    sns.kdeplot(x = potable[col], label = "Potable")
    plt.legend()
plt.tight_layout()

## Ploting Pairplot 
#### The Seaborn Pairplot allows us to plot pairwise relationships between variables within a dataset. This creates a nice visualisation and helps us understand the data by summarising a large amount of data in a single figure. 

In [ ]:
columns = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate',
           'Conductivity', 'Organic_carbon', 'Trihalomethanes',
           'Turbidity', 'Potability']
sns.pairplot(water, hue='ph', palette='coolwarm')
plt.show()

### Obesrving for missing values

In [ ]:
water.isnull().sum()

In [ ]:
msno.matrix(water)
plt.show()

### Handle missing value with average of features (mean value)

In [ ]:
print(water.columns)

In [ ]:
water['ph'].fillna(value = water['ph'].mean(), inplace = True)
water['Sulfate'].fillna(value = water['Sulfate'].mean(),inplace = True)
water['Trihalomethanes'].fillna(value = water['Trihalomethanes'].mean(),inplace = True)

In [ ]:
water.isnull().sum()

## Preprocessing: Train-Test Split and Normalization

In [ ]:
X = water.drop("Potability", axis=1).values
y = water["Potability"].values

#### Train Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 3)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

### Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X_train,y_train)
y_rfc = rfc.predict(X_test)

In [ ]:
accuracy_rfc = accuracy_score(y_test, y_rfc)
precision_rfc = precision_score(y_test, y_rfc, average='weighted')
recall_rfc = recall_score(y_test, y_rfc, average='weighted')
f1_rfc = f1_score(y_test, y_rfc, average='weighted')

print("Overall Accuracy:", accuracy_rfc)
print("Precision:", precision_rfc)
print("Recall:", recall_rfc)
print("F1 Score:", f1_rfc)

### Classification report 

In [ ]:
rfc_class_report = classification_report(y_rfc,y_test)
print(rfc_class_report)

### 2 -  Method using hyperparameter 'class_weight' to balance the label class (1)  To check Any imporovemt in Accuracy and other Matrix

In [ ]:
from sklearn.feature_selection import SelectFromModel

# Create and train the Random Forest classifier with adjusted class weights
rfc = RandomForestClassifier(class_weight='balanced', random_state=42)
rfc.fit(X_train, y_train)

# Get predicted probabilities of each class
y_pred_proba = rfc.predict_proba(X_test)

# Adjust the decision threshold (e.g., set it to 0.4)
threshold = 0.4
y_pred_1= (y_pred_proba[:, 1] >= threshold).astype(int)

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_1))

# Perform feature selection using Random Forest as the estimator
selector = SelectFromModel(rfc)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Train the Random Forest classifier on the selected features
rfc_selected = RandomForestClassifier(class_weight='balanced', random_state=42)
rfc_selected.fit(X_train_selected, y_train)

# Get predicted probabilities of each class using the selected features
y_pred_proba_selected = rfc_selected.predict_proba(X_test_selected)

# Adjust the decision threshold (e.g., set it to 0.4)
y_pred_selected = (y_pred_proba_selected[:, 1] >= threshold).astype(int)

accuracy_selected = accuracy_score(y_test, y_pred_selected)
precision_selected = precision_score(y_test, y_pred_selected)
recall_selected = recall_score(y_test, y_pred_selected)
f1_selected = f1_score(y_test, y_pred_selected)

# Print metrics for the model with feature selection
print("Model with Feature Selection:")
print("Overall Accuracy:", accuracy_selected)
print("Precision:", precision_selected)
print("Recall:", recall_selected)
print("F1 Score:", f1_selected)

#### Classification report Random Forest classifier with adjusted class weights

In [ ]:
# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_1))

print("Classification Report (with feature selection):")
print(classification_report(y_test, y_pred_selected))

### Random Search CV Alogorithm To check Any improvemt in Accuracy and other Matrix

#### Random Search:

##### Random search randomly samples the hyperparameter space by selecting a fixed number of random combinations of hyperparameters.It allows more flexibility by exploring a wider range of hyperparameter valuesRandom search is particularly effective when the hyperparameter search space is large or when the impact of different hyperparameters is not well understood.By randomly sampling hyperparameter values, it is more likely to cover a diverse range of combinations, which can lead to better performance compared to an exhaustive grid search. Random search is often more computationally efficient

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Define the parameter grid for random search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
# Perform random search
random_search = RandomizedSearchCV(rfc, param_grid, n_iter=10, cv=5)

# Fit the random search to training data
random_search.fit(X_train, y_train)

# Get the best estimator from the random search
best_rfc = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_rfc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the metrics
print("Overall Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


### Applying Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
y_gbc = gbc.predict(X_test)

accuracy_gbc = accuracy_score(y_test, y_gbc)
precision_gbc = precision_score(y_test, y_gbc, average='weighted')
recall_gbc = recall_score(y_test, y_gbc, average='weighted')
f1_gbc = f1_score(y_test, y_gbc, average='weighted')

print("Gradient Boosting - Overall Accuracy:", accuracy_gbc)
print("Precision:", precision_gbc)
print("Recall:", recall_gbc)
print("F1 Score:", f1_gbc)

### Classification report for RandomSearchCV

In [ ]:
print(classification_report(y_pred, y_test))

### Creating a DataFrame to store the results:

In [ ]:
results = pd.DataFrame({'Model': ['Random Forest','Model with Feature Selection','Random Search'],
                       'Accuracy': [accuracy_rfc, accuracy_selected, accuracy],
                       'Precision': [precision_rfc, precision_selected, precision],
                       'Recall': [recall_rfc, recall_selected, recall],
                       'F1-Score': [f1_rfc, f1_selected, f1]})


# Save the results to a CSV file
results.to_csv('classification_results.csv', index=False)

In [ ]:
results

# Saving the final model

In [ ]:
import pickle

# Save the trained Random Forest Classifier to a pickle file
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rfc, file)